![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL Readings

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

- C library (fast)
- lightweight disk-based database
- that doesn’t require a separate server process
- nonstandard variant of the SQL query language
- applications can use SQLite for internal data storage
- use case: prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle

[sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
!ls data

flights.db


In [3]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [6]:
print("""He said, "Hello, world!" The end""")

He said, "Hello, world!" The end


In [7]:
print('''He said, "Hello, world!" The end''')

He said, "Hello, world!" The end


In [11]:
cursor.execute("""SELECT name, type FROM sqlite_master WHERE type='table';""")
print(cursor.fetchall())

[('airports', 'table'), ('airlines', 'table'), ('routes', 'table')]


In [12]:
cursor.description

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

In [20]:
cursor.execute("SELECT name, type FROM sqlite_master")
#SAME AS
cursor.execute("select NAME, TYPE from SQLITE_MASTER")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [19]:
cursor.execute("SELECT type, tbl_name, rootpage FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 2)
('index', 'airports', 3)
('table', 'airlines', 945)
('index', 'airlines', 946)
('table', 'routes', 1393)
('index', 'routes', 1394)


In [18]:
cursor.description

(('type', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('tbl_name', None, None, None, None, None, None),
 ('rootpage', None, None, None, None, None, None),
 ('sql', None, None, None, None, None, None))

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [34]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [40]:
cursor.fetchone()

(6,
 '7',
 'Narsarsuaq',
 'Narssarssuaq',
 'Greenland',
 'UAK',
 'BGBW',
 '61.160517',
 '-45.425978',
 '112',
 '-3',
 'E',
 'America/Godthab')

In [49]:
for row in cursor.fetchmany(10):
    print('~~|~~'.join([str(x) for x in row]))

39~~|~~40~~|~~Clyde River~~|~~Clyde River~~|~~Canada~~|~~YCY~~|~~CYCY~~|~~70.486111~~|~~-68.516667~~|~~87~~|~~-5~~|~~A~~|~~America/Toronto
40~~|~~41~~|~~Fairmont Hot Springs~~|~~Coral Harbour~~|~~Canada~~|~~YZS~~|~~CYCZ~~|~~64.193333~~|~~-83.359444~~|~~2661~~|~~-5~~|~~A~~|~~America/Coral_Harbour
41~~|~~42~~|~~Dawson City~~|~~Dawson~~|~~Canada~~|~~YDA~~|~~CYDA~~|~~64.043056~~|~~-139.127778~~|~~1215~~|~~-8~~|~~A~~|~~America/Vancouver
42~~|~~43~~|~~Burwash~~|~~Burwash~~|~~Canada~~|~~YDB~~|~~CYDB~~|~~61.371111~~|~~-139.040556~~|~~2647~~|~~-8~~|~~A~~|~~America/Vancouver
43~~|~~44~~|~~Princeton~~|~~Princeton~~|~~Canada~~|~~YDC~~|~~CYDC~~|~~49.468056~~|~~-120.511389~~|~~2298~~|~~-8~~|~~A~~|~~America/Vancouver
44~~|~~45~~|~~Deer Lake~~|~~Deer Lake~~|~~Canada~~|~~YDF~~|~~CYDF~~|~~49.210833~~|~~-57.391388~~|~~72~~|~~-3.5~~|~~A~~|~~America/St_Johns
45~~|~~46~~|~~Dease Lake~~|~~Dease Lake~~|~~Canada~~|~~YDL~~|~~CYDL~~|~~58.422222~~|~~-130.032222~~|~~2600~~|~~-8~~|~~A~~|~~America/Vancouver
46~~|~~4

#### Use description

In [24]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [50]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [54]:
z = (1, 2)

In [53]:
x, y = (1, 2)
type(x)

int

In [58]:
print(z[0], z[1])

1 2


In [59]:
# same as
print(*z)

1 2


In [61]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep="\n"+'~'*15+'\n')

(0, 'index', 'INTEGER', 0, None, 0)
~~~~~~~~~~~~~~~
(1, 'id', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(2, 'name', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(3, 'alias', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(4, 'iata', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(5, 'icao', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(6, 'callsign', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(7, 'country', 'TEXT', 0, None, 0)
~~~~~~~~~~~~~~~
(8, 'active', 'TEXT', 0, None, 0)


In [27]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

### Constructing SQL queries

**`SELECT`**:  The columns you want
  - options: `DISTINCT`, using `AS` to rename columns, single number aggregates

**`FROM`:** the source tables
  - options: also uses aliasing with `AS`; joining with `[LEFT|INNER|RIGHT|FULL] JOIN ___ [ON|USING]`

**`WHERE`**: your filters
  - options: comparators like `=` & `>=`; `BETWEEN`, `IN`, `LIKE` (with wildcards `%`); booleans like `AND`, `OR`, `NOT`

**`ORDER BY`**: sorting
  - options: `ASC` (default) and `DESC`

**`LIMIT`**:  # of rows to return (pair with `OFFSET`)

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [63]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
 AND country = 'United Kingdom'
ORDER BY lower(name)
""")
cursor.fetchall()

[('AD Aviation',),
 ('Air Cudlua',),
 ('Air Foyle',),
 ('Air Southwest',),
 ('Air Wales',),
 ('All Europe',),
 ('Astraeus',),
 ('Aurigny Air Services',),
 ('BA CityFlyer',),
 ('BBN-Airways',),
 ('bmi',),
 ('bmibaby',),
 ('British Airways',),
 ('British International Helicopters',),
 ('British Mediterranean Airways',),
 ('British Midland Regional',),
 ('CB Airways UK ( Interliging Flights )',),
 ('Crest Aviation',),
 ('Eastern Airways',),
 ('easyJet',),
 ('Excel Airways',),
 ('Excel Charter',),
 ('First Choice Airways',),
 ('Flightline',),
 ('Flybe',),
 ('Flyglobespan',),
 ('GB Airways',),
 ('Highland Airways',),
 ('Jc royal.britannica',),
 ('Jet2.com',),
 ('Kinloss Flying Training Unit',),
 ('Monarch Airlines',),
 ('MyTravel Airways',),
 ('Norfolk County Flight College',),
 ('Rainbow Air Euro',),
 ('Royal European Airlines',),
 ('ScotAirways',),
 ('Thomas Cook Airlines',),
 ('Thomsonfly',),
 ('Virgin Atlantic Airways',)]

In [64]:
cursor.execute('''SELECT DISTINCT country FROM airports ORDER BY 1''')
print(*cursor.fetchall(), sep='\n')

('Afghanistan',)
('Albania',)
('Algeria',)
('American Samoa',)
('Angola',)
('Anguilla',)
('Antarctica',)
('Antigua and Barbuda',)
('Argentina',)
('Armenia',)
('Aruba',)
('Australia',)
('Austria',)
('Azerbaijan',)
('Bahamas',)
('Bahrain',)
('Bangladesh',)
('Barbados',)
('Belarus',)
('Belgium',)
('Belize',)
('Benin',)
('Bermuda',)
('Bhutan',)
('Bolivia',)
('Bosnia and Herzegovina',)
('Botswana',)
('Brazil',)
('British Indian Ocean Territory',)
('British Virgin Islands',)
('Brunei',)
('Bulgaria',)
('Burkina Faso',)
('Burma',)
('Burundi',)
('Cambodia',)
('Cameroon',)
('Canada',)
('Cape Verde',)
('Cayman Islands',)
('Central African Republic',)
('Chad',)
('Chile',)
('China',)
('Christmas Island',)
('Cocos (Keeling) Islands',)
('Colombia',)
('Comoros',)
('Congo (Brazzaville)',)
('Congo (Kinshasa)',)
('Cook Islands',)
('Costa Rica',)
("Cote d'Ivoire",)
('Croatia',)
('Cuba',)
('Cyprus',)
('Czech Republic',)
('Denmark',)
('Djibouti',)
('Dominica',)
('Dominican Republic',)
('East Timor',)
('Ecua

In [65]:
cursor.execute('''
SELECT country,
       count(*) AS airport_count
FROM airports
GROUP BY country
ORDER BY 1
''')
print(*cursor.fetchall(), sep='\n')

('Afghanistan', 21)
('Albania', 1)
('Algeria', 44)
('American Samoa', 3)
('Angola', 26)
('Anguilla', 1)
('Antarctica', 19)
('Antigua and Barbuda', 2)
('Argentina', 103)
('Armenia', 4)
('Aruba', 1)
('Australia', 263)
('Austria', 29)
('Azerbaijan', 11)
('Bahamas', 37)
('Bahrain', 2)
('Bangladesh', 13)
('Barbados', 1)
('Belarus', 9)
('Belgium', 34)
('Belize', 12)
('Benin', 1)
('Bermuda', 2)
('Bhutan', 1)
('Bolivia', 27)
('Bosnia and Herzegovina', 4)
('Botswana', 29)
('Brazil', 213)
('British Indian Ocean Territory', 1)
('British Virgin Islands', 2)
('Brunei', 3)
('Bulgaria', 7)
('Burkina Faso', 2)
('Burma', 43)
('Burundi', 1)
('Cambodia', 13)
('Cameroon', 10)
('Canada', 435)
('Cape Verde', 7)
('Cayman Islands', 3)
('Central African Republic', 3)
('Chad', 7)
('Chile', 38)
('China', 219)
('Christmas Island', 1)
('Cocos (Keeling) Islands', 1)
('Colombia', 72)
('Comoros', 4)
('Congo (Brazzaville)', 3)
('Congo (Kinshasa)', 36)
('Cook Islands', 8)
('Costa Rica', 29)
("Cote d'Ivoire", 7)
('Croat

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [75]:
cursor.execute("""
SELECT r.*,
      source.latitude AS source_lat,
	  source.longitude AS source_lon,
	  dest.latitude AS dest_lat,
	  dest.longitude AS dest_lon
	  
FROM routes r
JOIN airports source ON source.id = r.source_id
JOIN airports dest ON dest.id = r.dest_id
""")
df = pd.DataFrame(cursor.fetchall())

In [76]:
[x[0] for x in cursor.description]

['index',
 'airline',
 'airline_id',
 'source',
 'source_id',
 'dest',
 'dest_id',
 'codeshare',
 'stops',
 'equipment',
 'source_lat',
 'source_lon',
 'dest_lat',
 'dest_lon']

In [77]:
df.columns = [x[0] for x in cursor.description]

In [78]:
df

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,source_lat,source_lon,dest_lat,dest_lon
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2,43.449928,39.956589,55.606186,49.278728
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2,46.283333,48.006278,55.606186,49.278728
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2,46.283333,48.006278,44.225072,43.081889
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2,55.305836,61.503333,55.606186,49.278728
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2,55.305836,61.503333,55.012622,82.650656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67197,67658,ZL,4178,WYA,6334,ADL,3341,None,0,SF3,-33.0589,137.514,-34.945,138.530556
67198,67659,ZM,19016,DME,4029,FRU,2912,None,0,734,55.408611,37.906111,43.061306,74.477556
67199,67660,ZM,19016,FRU,2912,DME,4029,None,0,734,43.061306,74.477556,55.408611,37.906111
67200,67661,ZM,19016,FRU,2912,OSS,2913,None,0,734,43.061306,74.477556,40.608989,72.793269


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [85]:
# Which countries have the highest amount of active airports?
cursor.execute("""
SELECT
    country
  , count(*) AS airline_count
FROM airlines
WHERE active = 'Y'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [86]:
# Which countries have the highest amount of inactive airlines? -- NOTE: question change!
cursor.execute("""
SELECT
    country
  , count(*) AS airline_count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY count(*) DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [87]:
# What about airports by timezones?
cursor.execute("""
SELECT
    timezone AS tz
  , count(*) AS a_cnt
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


In [88]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [69]:
import pandas as pd

In [71]:
pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [72]:
df = pd.read_sql_query("""
SELECT r.*,
      source.latitude AS source_lat,
	  source.longitude AS source_lon,
	  dest.latitude AS dest_lat,
	  dest.longitude AS dest_lon
	  
FROM routes r
JOIN airports source ON source.id = r.source_id
JOIN airports dest ON dest.id = r.dest_id
""", pd_con)

In [79]:
df.index

RangeIndex(start=0, stop=67202, step=1)

In [ ]:
pd_con.close()

## Bonus: SQL-like querying in Pandas

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [89]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')

In [90]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()


,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
160,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
379,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
501,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
808,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


In [91]:
#vs
shelter_data[shelter_data['Name']=="Max"].head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
50,A736717,Max,10/21/2016 02:45:00 PM,10/21/2016 02:45:00 PM,10/16/2013,Return to Owner,NaN,Dog,Neutered Male,3 years,Pug Mix,Brown Brindle
160,A766116,Max,02/13/2018 11:14:00 AM,02/13/2018 11:14:00 AM,02/02/2015,Return to Owner,NaN,Dog,Neutered Male,3 years,Rottweiler/Staffordshire,Blue/Tan
379,A688196,Max,09/16/2014 12:20:00 PM,09/16/2014 12:20:00 PM,09/16/2005,Euthanasia,Suffering,Dog,Neutered Male,9 years,Jack Russell Terrier Mix,Chocolate/White
501,A579608,Max,07/21/2014 12:43:00 PM,07/21/2014 12:43:00 PM,06/09/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Black/White
808,A745151,Max,03/13/2017 07:12:00 PM,03/13/2017 07:12:00 PM,11/13/2016,Return to Owner,NaN,Dog,Neutered Male,3 months,German Shepherd Mix,White


In [92]:
# wrap columns with spaces in backticks
shelter_data.query('`Animal Type` == "Dog"').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A776359,Gizmo,07/18/2018 04:02:00 PM,07/18/2018 04:02:00 PM,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A720371,Moose,02/13/2016 05:59:00 PM,02/13/2016 05:59:00 PM,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
14,A765349,Einstein,06/08/2018 01:04:00 PM,06/08/2018 01:04:00 PM,01/18/2009,Adoption,Foster,Dog,Neutered Male,9 years,Chihuahua Shorthair Mix,Tricolor
15,A760697,Star,10/26/2017 03:22:00 PM,10/26/2017 03:22:00 PM,10/23/2007,Transfer,Partner,Dog,Intact Male,10 years,Yorkshire Terrier Mix,Brown/Black
16,A767231,Millie,02/25/2018 05:19:00 PM,02/25/2018 05:19:00 PM,02/25/2017,Return to Owner,NaN,Dog,Spayed Female,1 year,Jack Russell Terrier/Chihuahua Shorthair,White/Tan
